In [1]:
# maoer深度学习模型对比算法 -机器学习SVM&RF

import os
import pandas as pd
import numpy as np
import math
import copy
import datetime
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, recall_score, precision_score, roc_curve, confusion_matrix
from _collections import OrderedDict  # 导入 OrderedDict 来保持字典中键值对的顺序

print('||--------开始时间：', datetime.datetime.now(), '-------------')
data_time_windows_list = ['0101_0131', '0115_0215', '0201_0230','1101_1130', '1115_1215', '1201_1231', '1215_0115']
data_classifier_list = ['SVM']

# 参数设置
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2
num_heads = 10
feature_dim = 200
max_history_len = 15
num_experts = 3
num_tasks = 2
# 设置嵌入维度
continue_embedding_dim = 200
discrete_embedding_dim = 200
lr = 0.1
batch_size = 128
threshold = 0.5

# 获取时间窗内连续与离散特征名的列表
def get_continue_discrete_feature_namelist(time_windows, datapath):
    data = pd.read_csv(datapath)
    time_windows_data = data[(data['DataSet'] == time_windows)]
    user_history_pay_QOE_continue_column = eval([time_windows_data['QOE_continue'].values.tolist()][0][0])
    user_history_pay_CHONGHE_continue_column = eval([time_windows_data['CHONGHE_continue'].values.tolist()][0][0])
    user_history_pay_FUFEI_continue_column = eval([time_windows_data['FUFEI_continue'].values.tolist()][0][0])
    user_history_pay_QOE_discrete_column = eval([time_windows_data['QOE_discrete'].values.tolist()][0][0])
    user_history_pay_CHONGHE_discrete_column = eval([time_windows_data['CHONGHE_discrete'].values.tolist()][0][0])
    user_history_pay_FUFEI_discrete_column = eval([time_windows_data['FUFEI_discrete'].values.tolist()][0][0])


    return user_history_pay_QOE_continue_column, user_history_pay_CHONGHE_continue_column,user_history_pay_FUFEI_continue_column,\
            user_history_pay_QOE_discrete_column,user_history_pay_CHONGHE_discrete_column,user_history_pay_FUFEI_discrete_column



||--------开始时间： 2024-03-18 01:14:35.557532 -------------


In [2]:
# 1.数据处理+划分训练、验证、测试集

# 划分数据集 给定输出后固定结果 输出形式定为存储user_id 形成train_dataset,val_dataset,test_dataset
def split_data_unique(input_file, output_file, train_ratio, val_ratio, test_ratio):
    df = pd.read_csv(input_file)
#     data = df[df.columns[0]].unique()  # 提取第一列数据并去重
    data = np.arange(df.shape[0])  #一个从 0 到行数减 1 的整数数组

    np.random.shuffle(data)  # 随机打乱数据
    # 划分数据
    total_len = len(data)
    x_end = int(total_len * train_ratio)
    y_end = x_end + int(total_len * val_ratio)
    train_data = data[:x_end]
    val_data = data[x_end:y_end]
    test_data = data[y_end:]
    # 存储结果是去重的user_id
    result = {
        'Train': train_data,
        'Val': val_data,
        'Test': test_data
    }   
    # 创建每个子集的DataFrame  
    train_df = pd.DataFrame(train_data, columns=['Train'])
    val_df = pd.DataFrame(val_data, columns=['Val'])
    test_df = pd.DataFrame(test_data, columns=['Test'])
    # 将每个DataFrame转换为一列Series  
    train_series = train_df.iloc[:, 0]
    val_series = val_df.iloc[:, 0]
    test_series = test_df.iloc[:, 0]
    # 为了确保所有Series有相同的长度，我们需要找到最大长度并截断较短的Series  
    max_len = max(len(train_series), len(val_series), len(test_series))
    train_series = train_series.reindex(range(max_len)).fillna(value=pd.NA)
    val_series = val_series.reindex(range(max_len)).fillna(value=pd.NA)
    test_series = test_series.reindex(range(max_len)).fillna(value=pd.NA)
    # 创建一个新的DataFrame，将Series作为列  
    combined_df = pd.DataFrame({
        'Train': train_series,
        'Val': val_series,
        'Test': test_series
    })
    # 写入CSV文件，不包含索引和列名  
    combined_df.to_csv(output_file, index=False)
    print('已输出划分数据集结果')

# 数据预处理 MachineLearning特有修改版 离散值中的空值填充
def data_pre_deal(input_path,continue_feature_list,discrete_feature_list,label_feature):
    df = pd.read_csv(input_path)
    deal_data_df = [] # 待修改********
    discrete_cols_to_fill = [col for col in df.columns if col in discrete_feature_list]
    # 离散值 使用出现次数最多的非空值填充空值
    most_common_values = df[discrete_cols_to_fill].mode().iloc[0]
    df[discrete_cols_to_fill] = df[discrete_cols_to_fill].fillna(most_common_values)
    # 连续值 均值填充
    # new_df[continue_cols_to_fill] = new_df[continue_cols_to_fill].fillna(new_df[continue_cols_to_fill].mean())
    # print(len(discrete_cols_to_fill)+len(continue_cols_to_fill))
    feature_list = discrete_feature_list+continue_feature_list+label_feature
    new_df_cols = [col for col in df.columns if col in feature_list]
    new_df = df[new_df_cols]
    print('数据预处理结束')
    return new_df


# 改过 根据 BiLSTM单独改的 根据下标获取数据
def find_data_by_list(index_list, intput_data_df):  
    data_list = []  
#     print('index_list',len(index_list),intput_data_df.shape)
    for index in index_list:  
        # index_list必须是列表 最后result_df才能是DataFrame类型
        index = int(index)
        index_list = []
        index_list.append(index)
        data_list.append(intput_data_df.iloc[index_list])
    result_df = pd.concat(data_list, axis=0, ignore_index=True)  
#     print('result_df',type(result_df),result_df.shape)
    return result_df

    
# 获取列唯一值数量表，并对离散特征的值转化为从0开始的索引
def get_unique_feature_num_and_discrete_valueChange(datadf,discrete_feature_column_list):
    # 获取离散特征的类别数量，并存储为字典
    feature_category_num_dict = {}
    for column in datadf.columns:
        unique_values_len = datadf[column].nunique()  # 获取列的唯一值数量
        feature_category_num_dict[column] = unique_values_len
        if column in discrete_feature_column_list:
            unique_values = datadf[column].unique()
            value_mapping_dict = {value: index for index, value in enumerate(unique_values) if
                              value != -1 and value != '' and value is not None}
            datadf[column] = datadf[column].map(value_mapping_dict)
    return feature_category_num_dict,datadf

# 总的特征输入，生成划分后数据集及其输入   ML特有修改
def data_input(data_time_windows, path, spilt_outpath, train_ratio, val_ratio, test_ratio, feature_column_dict):
    dataset_path = path  
    dataset_spilt_path = spilt_outpath  
    if os.path.exists(dataset_spilt_path):  # 划分训练、验证、测试集
        print("划分文件已存在，不再进行数据划分")
    else:
        split_data_unique(dataset_path, dataset_spilt_path, train_ratio, val_ratio, test_ratio)
    deal_data_df = data_pre_deal(dataset_path, feature_column_dict['total_continue_feature'], feature_column_dict['total_discrete_feature'],feature_column_dict['D'])  # 数据预处理
    # 获取离散特征的类别数量，并存储为字典
    feature_category_num_dict,deal_data_df  = get_unique_feature_num_and_discrete_valueChange(deal_data_df, feature_column_dict['total_discrete_feature'])
    # 读取划分文件的结果
    spilt_data_df = pd.read_csv(dataset_spilt_path)
    # 输出每一列数据为列表
    train_list = spilt_data_df['Train'].tolist()
    val_list = spilt_data_df['Val'].tolist()
    test_list = spilt_data_df['Test'].tolist()
    train_list = [x for x in train_list if not math.isnan(x)]
    val_list = [x for x in val_list if not math.isnan(x)]
    test_list = [x for x in test_list if not math.isnan(x)]

    train_data = find_data_by_list(train_list, deal_data_df).copy()
    val_data = find_data_by_list(val_list, deal_data_df).copy()
    test_data = find_data_by_list(test_list, deal_data_df).copy()
    print(type(train_data),train_data.shape)  # 检查是否是DataFrame类型  
    # 取label
    train_label = train_data.iloc[:,-1]
    val_label = val_data.iloc[:,-1]
    test_label = test_data.iloc[:,-1]
    # 1/2 转为 0/1
    train_label,_ = map_labels_to_int(train_label)
    val_label,_ = map_labels_to_int(val_label)
    test_label,_ = map_labels_to_int(test_label)
    # 取除label外的其他列
    train_data = train_data.iloc[:, :-1]
    val_data = val_data.iloc[:, :-1]
    test_data = test_data.iloc[:, :-1]
#     print('train_data',train_data.shape)
#     print('train_label',train_label.shape)
    print('数据划分完成')
    return train_data, val_data, test_data,train_label,val_label,test_label, feature_category_num_dict

# # 自定义数据加载器
# class DynamicDataLoader(DataLoader):
#     def __init__(self, dataset, batch_size):
#         super().__init__(dataset, batch_size, shuffle=False, drop_last=True)

#     def __iter__(self):
#         for batch in super().__iter__():
#             yield batch
#             # yield [{key: sample[key] for key in sample} for sample in batch]

# 自动评估阈值，计算ACC 、 Precision 等评估指标
def evaluate(y_true, y_pred, digits=4, cutoff='auto'):
    '''
    Args:
        y_true: list, labels, y_pred: list, predictions, digits: The number of decimals to use when rounding the number. Default is 4（保留小数后几位）
        cutoff: float or 'auto'
    Returns:
        evaluation: dict
    '''
    # 根据预测概率值y_pred计算最佳的切分阈值
    if cutoff == 'auto':
        fpr, tpr, thresholds = roc_curve(y_true, y_pred)
        youden = tpr-fpr
        cutoff = thresholds[np.argmax(youden)]
    y_pred_t = [1 if i > cutoff else 0 for i in y_pred]

    evaluation = OrderedDict()
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred_t).ravel()
    evaluation['auc'] = round(roc_auc_score(y_true, y_pred), digits)
    evaluation['acc'] = round(accuracy_score(y_true, y_pred_t), digits)
    evaluation['recall'] = round(recall_score(y_true, y_pred_t), digits)
    evaluation['precision'] = round(precision_score(y_true, y_pred_t, zero_division=1), digits)
    evaluation['specificity'] = round(tn / (tn + fp), digits)
    evaluation['F1'] = round(f1_score(y_true, y_pred_t), digits)
    evaluation['cutoff'] = cutoff

    return evaluation

# 选择分类器
def classifier_chose(classifier_name):
    clf = None
    if classifier_name =='SVM':
        clf = svm.SVC(C=1.0, kernel='rbf', decision_function_shape='ovr', probability=True)
    elif classifier_name =='RF':
        clf = RandomForestClassifier(n_estimators=100, random_state=42)
    else:
        print('分类器选择错误，仅 SVM / RF')
    return clf
    
# 标签值映射到从0开始的整数上
def map_labels_to_int(labels):
    label_map = {}  # 创建空字典用于存储标签值到整数的映射关系
    mapped_labels = []  # 存储映射后的整数标签列表
    count = 0  # 用于生成从 0 开始的整数

    for label in labels:
        if label not in label_map:
            label_map[label] = count
            count += 1
        mapped_labels.append(label_map[label])
    mapped_labels_series = pd.Series(mapped_labels)
    return mapped_labels_series, label_map  # 映射后的整数标签列表，标签映射关系

In [3]:
# 模型运行
# 2个分类器与7个时间窗
for data_classifier in data_classifier_list:
    for data_time_windows in data_time_windows_list:
        path = './Dataset/' + data_time_windows + '_user_pay_pred_feature_deal.csv'
        dataset_spilt_path = './Dataset/' + data_time_windows + '_user_pay_pred_feature_spilt_BiLSTM.csv'
    #     output_weight_result_path = './Dataset/' + data_time_windows + '_user_pay_pred_result_weight.csv'
        data_feature_continue_discrete_namelist_path = './Dataset/maoer_timewindows_continue_discrete_feature_column.csv'    # 连续与离散划分表
        # 获取时间窗内连续与离散特征名的列表
        user_history_pay_QOE_continue_column, user_history_pay_CHONGHE_continue_column, \
        user_history_pay_FUFEI_continue_column, user_history_pay_QOE_discrete_column,\
        user_history_pay_CHONGHE_discrete_column, user_history_pay_FUFEI_discrete_column = get_continue_discrete_feature_namelist(data_time_windows, data_feature_continue_discrete_namelist_path)
        user_feature_continue_column = []
        user_feature_discrete_column = []
        # total continue feature
        total_continue_feature = user_feature_continue_column+user_history_pay_QOE_continue_column+user_history_pay_CHONGHE_continue_column+user_history_pay_FUFEI_continue_column
        total_discrete_feature = user_feature_discrete_column+user_history_pay_QOE_discrete_column+user_history_pay_CHONGHE_discrete_column+user_history_pay_FUFEI_discrete_column
        total_discrete_feature_add_D = user_feature_discrete_column+user_history_pay_QOE_discrete_column+user_history_pay_CHONGHE_discrete_column+user_history_pay_FUFEI_discrete_column
        total_discrete_feature_add_D.append('user_in_drama_is_pay_for_drama_in_next_time')
        D = ['user_in_drama_is_pay_for_drama_in_next_time']
        tensor_dict_idx = ['pay_QOE_continue','pay_QOE_discrete','pay_CHONGHE_continue','pay_CHONGHE_discrete','pay_FUFEI_continue','pay_FUFEI_discrete','target_QOE_continue','target_QOE_discrete','target_CHONGHE_continue','target_CHONGHE_discrete','target_FUFEI_continue','target_FUFEI_discrete']
        # print(len(user_history_pay_QOE_continue_column),len(user_history_pay_CHONGHE_continue_column),len(user_history_pay_FUFEI_continue_column))
        # 形成对应需要的特征名称列表
        feature_column_dict = {
            'total_continue_feature': total_continue_feature,
            'total_discrete_feature': total_discrete_feature,
            'D':D
        }
        # 创建一个空的DataFrame来存储结果
        test_auc_df = pd.DataFrame(columns=['时间','model','运行位置','Type','dataset','train_ratio','feature_embedding','batchSize','lr','max_history_len','实验数', '测试集总损失', 'AUC','ACC','F1','Precision','Recall'])
        
        # 获取数据
        train_data, val_data, test_data,\
        train_label,val_label,test_label, feature_category_num_dict = data_input(data_time_windows, path, dataset_spilt_path,
                                                                         train_ratio, val_ratio, test_ratio, 
                                                                         feature_column_dict)    
        # 将验证数据集val_data加入train_data
        train_data = pd.concat([train_data, val_data], axis=0)
        train_label = pd.concat([train_label, val_label], axis=0)
#         train_data = train_data+val_data
#         # 五折交叉
#         kf = KFold(n_splits=10,shuffle=True)  # 初始化KFold
#         train_files = []   # 存放10折的训练集划分
#         test_files = []     # # 存放10折的测试集集划分
        total_auc_test = 0.0
        total_acc_test  = 0
        total_f1_test = 0
        total_precision_test = 0
        total_recall_test = 0
        test_time = 0
        test_label_len = 0
        
        for i in range(5):
            print(f"i=:{i+1}")
            n = i
            
            # SVM
            clf = classifier_chose(data_classifier)
            clf.fit(train_data, train_label)
            y_prob = clf.predict_proba(test_data)
            y_prob = y_prob[:, 0]
            evaluation = evaluate(test_label, y_prob)
            total_acc_test += evaluation['acc']
            total_f1_test += evaluation['F1']
            total_recall_test += evaluation['recall']
            total_precision_test += evaluation['precision']
            total_auc_test += evaluation['auc']
            test_time += 1
            print('||--------当前轮次:',i,'结束时间：',datetime.datetime.now(),'-------------')

        # 平均损失
#             average_loss_test = total_loss_test / len(test_label_len)
        average_auc_test = total_auc_test / 5
        average_acc_test = total_acc_test / 5
        average_f1_test = total_f1_test / 5
        average_precision_test = total_precision_test / 5
        average_recall_test = total_recall_test / 5
        print(
            f"Test AUC: {average_auc_test},ACC:{average_acc_test},F1:{average_f1_test},Precision:{average_precision_test},Recall:{average_recall_test}")

        # 将本次训练的结果添加到DataFrame中
        test_auc_df = test_auc_df.append({'时间':datetime.datetime.now(),'model':'MachineLearning-SVM','运行位置':'CPU','Type':'Origin','dataset':data_time_windows,'train_ratio':train_ratio,'feature_embedding':0,'batchSize':batch_size,'lr':0,'max_history_len':0,'实验数': i + 1,'测试集总损失': 0, 'AUC': average_auc_test,'ACC': average_acc_test,'F1': average_f1_test,'Precision': average_precision_test,'Recall': average_recall_test}, ignore_index=True)
        # 将结果保存到CSV文件中
        with open('./Dataset/maoerDL_result_maoer_pay_pred_model3_1.csv', 'a') as f:
            test_auc_df.to_csv(f, index=False)
    #     with open('./Dataset/maoerDL_result_maoer_pay_pred_weight_model3_1.csv', 'a') as f:
    #         test_weight_df.to_csv(f, index=False)

        print('结果已输出')
        print('||--------当前时间窗',data_time_windows,'结束时间：',datetime.datetime.now(),'-------------')

划分文件已存在，不再进行数据划分
数据预处理结束
<class 'pandas.core.frame.DataFrame'> (13696, 84)
数据划分完成
i=:1
||--------当前轮次: 0 结束时间： 2024-03-18 01:21:10.533323 -------------
i=:2
||--------当前轮次: 1 结束时间： 2024-03-18 01:26:34.757774 -------------
i=:3
||--------当前轮次: 2 结束时间： 2024-03-18 01:31:55.644580 -------------
i=:4
||--------当前轮次: 3 结束时间： 2024-03-18 01:37:14.881748 -------------
i=:5
||--------当前轮次: 4 结束时间： 2024-03-18 01:42:34.978351 -------------
Test AUC: 0.63138,ACC:0.628,F1:0.5142,Precision:0.5629,Recall:0.4732
结果已输出
||--------当前时间窗 0101_0131 结束时间： 2024-03-18 01:42:34.994195 -------------
划分文件已存在，不再进行数据划分
数据预处理结束
<class 'pandas.core.frame.DataFrame'> (12096, 79)
数据划分完成
i=:1
||--------当前轮次: 0 结束时间： 2024-03-18 01:47:16.851563 -------------
i=:2
||--------当前轮次: 1 结束时间： 2024-03-18 01:51:24.461840 -------------
i=:3
||--------当前轮次: 2 结束时间： 2024-03-18 01:55:26.041233 -------------
i=:4
||--------当前轮次: 3 结束时间： 2024-03-18 01:59:29.878888 -------------
i=:5
||--------当前轮次: 4 结束时间： 2024-03-18 02:03:27.142600 --